<a href="https://colab.research.google.com/github/mushrafi88/asr_bangla/blob/main/t5_asr_bn_corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Install libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install huggingface
!pip install datasets tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install wandb
!pip install gdown
!pip install pyarrow 
!pip install fastparquet
!pip install pandarallel 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-pyth

In [3]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from tqdm import tqdm
import datasets

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    TrainingArguments, Trainer, DataCollatorForSeq2Seq,AutoTokenizer,FlaxT5ForConditionalGeneration,AutoModelForSeq2SeqLM
  )

from torch.utils.data import Dataset, DataLoader

In [5]:
pd.set_option('display.max_colwidth', None)

In [7]:
df_custom = pd.read_parquet('/content/drive/MyDrive/data_wav2vec2/custom_data.parquet')
df_wav2vec2_all = pd.read_parquet('/content/drive/MyDrive/data_wav2vec2/wav2vec2_all.parquet')
df_wav2vec2_masked = pd.read_parquet('/content/drive/MyDrive/data_wav2vec2/wav2vec2_masked.parquet')

In [8]:
df_custom

,sentence,masked_sen,error_word,correct_word,lavenshtein_distance
0,আর জামিনের আবেদন করলেন না অর্পিতা,আর জামিনের [MASK] করলেন না অর্পিতা,সবেদন,আবেদন,20
1,অর্পিতা মুখার্জি কারাগারেই,অর্পিতা মুখার্জি [MASK],কারাগ ক্ষ্ম রএই,কারাগারেই,53
2,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জী ন্ধ ন,জীবন,62
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন খারিজ করে তাঁকে দিনের জেল হেফাজতে পাঠান,আদালত শুনানি শেষে পার্থের জামিন আবেদন [MASK] করে তাঁকে দিনের জেল হেফাজতে পাঠান,খা র্ধ িজ,খারিজ,44
4,অর্পিতাকেও দিনের জেল হেফাজতে রাখার আদেশ দেন,অর্পিতাকেও দিনের জেল [MASK] রাখার আদেশ দেন,হে গ্ম াজতে,হেফাজতে,45
...,...,...,...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম আলোকে বলেন,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম [MASK] বলেন,আলোযে,আলোকে,20
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে আটক,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে [MASK],আলোযে,আটক,80
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি অভিযোগ আনা হয়,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি অভিযোগ [MASK] হয়,আলোযে,আনা,80
2992537,পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পঁচটি,পাঁচটি,16


In [9]:
def mask_replace(sen,error_word):
  sen = sen.split()
  masked_index = sen.index("[MASK]")
  sen[masked_index] = error_word
  return " ".join(sen)

In [10]:
sen , error_word =  df_custom['masked_sen'][0], df_custom['error_word'][0]
print(sen)
print(error_word)

আর জামিনের [MASK] করলেন না অর্পিতা
সবেদন


In [11]:
mask_replace(sen,error_word)

'আর জামিনের সবেদন করলেন না অর্পিতা'

In [12]:
df_custom['error_sentence'] = df_custom.parallel_apply(lambda x: mask_replace(x.masked_sen,x.error_word),axis=1)

In [13]:
df_custom = df_custom[['error_sentence','sentence']]
df_custom.rename(columns = {'error_sentence':'input','sentence':'output'},inplace=True)
df_custom

<ipython-input-13-52c58ce796ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_custom.rename(columns = {'error_sentence':'input','sentence':'output'},inplace=True)


,input,output
0,আর জামিনের সবেদন করলেন না অর্পিতা,আর জামিনের আবেদন করলেন না অর্পিতা
1,অর্পিতা মুখার্জি কারাগ ক্ষ্ম রএই,অর্পিতা মুখার্জি কারাগারেই
2,তাঁর জী ন্ধ ন সংশয়ের আশঙ্কা রয়েছে,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন খা র্ধ িজ করে তাঁকে দিনের জেল হেফাজতে পাঠান,আদালত শুনানি শেষে পার্থের জামিন আবেদন খারিজ করে তাঁকে দিনের জেল হেফাজতে পাঠান
4,অর্পিতাকেও দিনের জেল হে গ্ম াজতে রাখার আদেশ দেন,অর্পিতাকেও দিনের জেল হেফাজতে রাখার আদেশ দেন
...,...,...
2992534,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম আলোযে বলেন,রায়ের পর প্রসিকিউটর রেজিয়া সুলতানা প্রথম আলোকে বলেন
2992535,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে আলোযে,খলিলুরের বিরুদ্ধে মুক্তিযুদ্ধকালে আটক
2992536,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি অভিযোগ আলোযে হয়,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি অভিযোগ আনা হয়
2992537,পঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে


In [14]:
df_wav2vec2_masked = df_wav2vec2_masked[['wav2vec2','sentence']]
df_wav2vec2_masked.rename(columns={'wav2vec2':'input','sentence':'output'},inplace=True)
df_wav2vec2_masked

<ipython-input-14-2bf526bdf7f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wav2vec2_masked.rename(columns={'wav2vec2':'input','sentence':'output'},inplace=True)


,input,output
0,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে
1,শাক দেশ গুরুতে,সার্ক দেশগুলোতে
2,শহজি জাদে,সহজেই যাওয়া যায়
3,শহজি জাদে,সহজেই যাওয়া যায়
4,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্তও করেছে রাশিয়া
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেযা পেরায়সি অতি আলোকীয় গতি পদর্শন করে,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে যা প্রায়শই অতিআলোকীয় গতি পশ্রন করে
248770,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠক ছিলেন,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠক ছিলেন
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ্রেস পার্থি খড়কপুর সদর বিধানসভা কেন্দ্রের বিধায়ক হিসেবে নির্বাচিত হন,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ্রেস পথ্রী খড়গপুর সদর বিধানসভা কেন্দ্রের বিধায়ক হিসেবে নির্বাচিত হন
248772,এখানে ল্বযাট ব্যাং্ক পরিষেবা বর্তমান,এখানে ল্বাড ব্যাঙ্ক পরিষেবা বর্তমান


In [15]:
df=df_custom.append(df_wav2vec2_masked)

<ipython-input-15-c46140c12b4a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_custom.append(df_wav2vec2_masked)


In [16]:
df = df.dropna(how='any',axis=0) 
df

,input,output
0,আর জামিনের সবেদন করলেন না অর্পিতা,আর জামিনের আবেদন করলেন না অর্পিতা
1,অর্পিতা মুখার্জি কারাগ ক্ষ্ম রএই,অর্পিতা মুখার্জি কারাগারেই
2,তাঁর জী ন্ধ ন সংশয়ের আশঙ্কা রয়েছে,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে
3,আদালত শুনানি শেষে পার্থের জামিন আবেদন খা র্ধ িজ করে তাঁকে দিনের জেল হেফাজতে পাঠান,আদালত শুনানি শেষে পার্থের জামিন আবেদন খারিজ করে তাঁকে দিনের জেল হেফাজতে পাঠান
4,অর্পিতাকেও দিনের জেল হে গ্ম াজতে রাখার আদেশ দেন,অর্পিতাকেও দিনের জেল হেফাজতে রাখার আদেশ দেন
...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেযা পেরায়সি অতি আলোকীয় গতি পদর্শন করে,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে যা প্রায়শই অতিআলোকীয় গতি পশ্রন করে
248770,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠক ছিলেন,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠক ছিলেন
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ্রেস পার্থি খড়কপুর সদর বিধানসভা কেন্দ্রের বিধায়ক হিসেবে নির্বাচিত হন,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ্রেস পথ্রী খড়গপুর সদর বিধানসভা কেন্দ্রের বিধায়ক হিসেবে নির্বাচিত হন
248772,এখানে ল্বযাট ব্যাং্ক পরিষেবা বর্তমান,এখানে ল্বাড ব্যাঙ্ক পরিষেবা বর্তমান


In [17]:
df = df.sample(3241313)

In [18]:
df = df.reset_index(drop=True)
df

,input,output
0,একটু বড় ধরনের ইনপুটের জন্য টাইপের সমস্যার সাধঃন করাটা একরকম অসম্ভব,একটু বড় ধরনের ইনপুটের জন্য টাইপের সমস্যার সমাধান করাটা একরকম অসম্ভব
1,ডমেনিকান প্রজাতন্ত্রের ত্তর উপকলে অ্যাটল্যান্টিক মহাসাগর এবং দক্ষিণে ক্যারেবীয় সাগর,ডোমিনিকান প্রজাতন্ত্রের উত্তর উপকূলে আটলান্টিক মহাসাগর এবং দক্ষিণে ক্যারিবীয় সাগর
2,কিশোরকিশোরী যেমন অনেক বেশি ৎসাহ পায়,কিশোরকিশোরী যেমন অনেক বেশি উৎসাহ পায়
3,রাতে হাজি সেলিমের বছরের কারাদণ্ড হয়,রায়ে হাজি সেলিমের বছরের কারাদণ্ড হয়
4,যদি কাছে মাসের দীর্ঘ মৌসুম না করা যায়,যদি চার মাসের দীর্ঘ মৌসুম না করা যায়
...,...,...
3241308,ফেব্রুয়ারির শেষ দিকে যুদ্ধ শুরু হওয়ার পর তাদের হাজার সেনা নিহত হুেছে য্য,ফেব্রুয়ারির শেষ দিকে যুদ্ধ শুরু হওয়ার পর তাদের হাজার সেনা নিহত হয়েছেন
3241309,সালের ডিসেম্বর সালের শিল্পনীতি সংশোধন করা সময়ে,সালের ডিসেম্বর সালের শিল্পনীতি সংশোধন করা হয়
3241310,দুর্ভিক্ষের ম্ ণ্য য়োজন নেই,দুর্ভিক্ষের প্রয়োজন নেই
3241311,তাহলে তো আপনাদের মতো নীতিনির্ধারকদের ভাবার বিষয়,সে তো আপনাদের মতো নীতিনির্ধারকদের ভাবার বিষয়


In [20]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [21]:
#df = df.rename(columns={"wav2vec2": "input", "sentence": "output"})

In [22]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((2917181, 2), (324132, 2))

In [23]:
test_df.head()

,input,output
292730,বুলগেরীয় ও ম্যাচটোনীয় ভাষার অনেকগুলো বৈশষ বৈশিষ্ট্য এগুলোকে অন্যান্য সব স্লাবী ভাষা থেকে স্বতন্ড করেছে,বুলগেরীয় ও ম্যাসডোনীয় ভাষার অনেকগুলি বৈশিষ্ট্য এগুলিকে অন্যান্য সব স্লাভীয় ভাষা থেকে স্বতন্ত্র করেছে
642973,সরকারি তথ্য নুযায়ী,সরকারি তথ্য অনুযায়ী
1062094,শিশুরা কোথায় ঝেলবে,শিশুরা কোথায় খেলবে
124263,দুর্বৃত্তরা পরিকল্পিতভাবে আগুন ধরিয়অ্যা দিয়েছে,দুর্বৃত্তরা পরিকল্পিতভাবে আগুন ধরিয়ে দিয়েছে
3197032,খানিকটা কৌতূহল থেকেই বাজারে রকজনকে জিজ্ঞেস করলাম,খানিকটা কৌতূহল থেকেই বাজারে একজনকে জিজ্ঞেস করলাম


In [24]:
tokenizer(df.input[30542])

{'input_ids': [1963, 735, 439, 492, 26, 6, 8059, 460, 9819, 3, 40, 739, 120, 17, 44, 6020, 1201, 456, 2060, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
#test_df['input_token_len'].describe()

### We will use a token length of 64 since it will cover the vast majority of examples

In [26]:
train_dataset = Dataset.from_pandas(train_df.sample(2543967))
test_dataset = Dataset.from_pandas(test_df)

In [27]:
train_dataset.save_to_disk('/content/train')
test_dataset.save_to_disk('/content/test')

Saving the dataset (0/2 shards):   0%|          | 0/2543967 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/324132 [00:00<?, ? examples/s]

In [28]:
train_dataset[1]

{'input': 'তারা  র্দ নেকটাে সাবধানতার সঙ্গে খাবার খেতে হয়',
 'output': 'তারা অনেকটাই সাবধানতার সঙ্গে খাবার খেতে হয়',
 '__index_level_0__': 2593555}

### Process Dataset

In [29]:
max_len = 64 

In [30]:
train_dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 2543967
})

In [31]:
tokenizer(train_dataset['output'][0])

{'input_ids': [56, 203, 163, 238, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [32]:
def prepare_data(example):
  example["input_ids"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).input_ids
  example["attention_mask"] = tokenizer(example['input'], padding="max_length", max_length=64,return_attention_mask = True).attention_mask
  example["labels"] = tokenizer(example["output"], padding="max_length", max_length=64).input_ids
  return example

In [33]:
train_dataset[0]

{'input': 'তিনি মানুশো হিসেবে ভালো',
 'output': 'তিনি মানুষ হিসেবে ভালো',
 '__index_level_0__': 554720}

In [34]:
train_data = train_dataset.map(prepare_data,remove_columns=train_dataset.column_names)

  0%|          | 0/2543967 [00:00<?, ?ex/s]

In [40]:
test_data = test_dataset.map(prepare_data)

  0%|          | 0/324132 [00:00<?, ?ex/s]

In [42]:
%mkdir /content/drive/MyDrive/data_wav2vec2/data_sharded

In [43]:
train_data.save_to_disk('/content/drive/MyDrive/data_wav2vec2/data_sharded/train')
test_data.save_to_disk('/content/drive/MyDrive/data_wav2vec2/data_sharded/test')

Saving the dataset (0/5 shards):   0%|          | 0/2543967 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/324132 [00:00<?, ? examples/s]

# Load data from here

In [29]:
train_data = datasets.Dataset.load_from_disk('/content/drive/MyDrive/data_wav2vec2/data_sharded/train')
test_data = datasets.Dataset.load_from_disk('/content/drive/MyDrive/data_wav2vec2/data_sharded/test')

In [30]:
train_data.cleanup_cache_files()
test_data.cleanup_cache_files()

2

In [31]:
model_name = 'flax-community/bengali-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--flax-community--bengali-t5-base/snapshots/e27fdf4c9c55d7c6e12df9ee4209eb2e3c1cd4ba/config.json
Model config MT5Config {
  "_name_or_path": "flax-community/bengali-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_

In [32]:
test_data

Dataset({
    features: ['input', 'output', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 324132
})

In [33]:
deta = train_data.train_test_split(0.25)
train_data = deta['train']
#test_data = deta['test']
duta = test_data.train_test_split(0.25)
test_data = duta['train']

In [34]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1907975
})

In [35]:
test_data

Dataset({
    features: ['input', 'output', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 243099
})

In [36]:
tokenizer.decode(train_data[0]['labels'])

'এখানে আওয়ামী লীগের নেতৃত্ব দেওয়ার মতো যোগ্য কোনো লোক নাই</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

### Train Model

In [37]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [38]:
%mkdir -p /content/drive/MyDrive/T5_model_raw_sentence/weights

In [39]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [41]:
# defining training related arguments
batch_size = 32
args = TrainingArguments(output_dir="mushrafi88/T5-asr-corrector",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=1e-5,
                        num_train_epochs=2,
                        weight_decay=0.01,
                        save_total_limit=2,
                        fp16 = True,
                        gradient_accumulation_steps = 4,
                        eval_steps = 5000,
                        save_steps = 5000,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        push_to_hub=True,
                       )
                                         

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [42]:
# defining trainer using 🤗
trainer = Trainer(model=model, 
                args=args, 
                train_dataset= train_data,
                eval_dataset=test_data,
                tokenizer=tokenizer,
                data_collator=data_collator)

/content/mushrafi88/T5-asr-corrector is already a clone of https://huggingface.co/mushrafi88/T5-asr-corrector. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [43]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1907975
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 29812
  Number of trainable parameters = 247539456
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
5000,0.199100,0.155570


The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, input, output. If __index_level_0__, input, output are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 243099
  Batch size = 32
Saving model checkpoint to mushrafi88/T5-asr-corrector/checkpoint-5000
Configuration saved in mushrafi88/T5-asr-corrector/checkpoint-5000/config.json
Configuration saved in mushrafi88/T5-asr-corrector/checkpoint-5000/generation_config.json
Model weights saved in mushrafi88/T5-asr-corrector/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in mushrafi88/T5-asr-corrector/checkpoint-5000/tokenizer_config.json
Special tokens file saved in mushrafi88/T5-asr-corrector/checkpoint-5000/special_tokens_map.json
tokenizer config file saved in mushrafi88/T5-asr-corrector/tokenizer_config.json
Special tokens fi

In [ ]:
trainer.save_model('t5_asr_bn_corrector')

Saving model checkpoint to t5_asr_bn_corrector
Configuration saved in t5_asr_bn_corrector/config.json
Model weights saved in t5_asr_bn_corrector/pytorch_model.bin
tokenizer config file saved in t5_asr_bn_corrector/tokenizer_config.json
Special tokens file saved in t5_asr_bn_corrector/special_tokens_map.json
Saving model checkpoint to mushrafi88/T5-asr-corrector
Configuration saved in mushrafi88/T5-asr-corrector/config.json
Model weights saved in mushrafi88/T5-asr-corrector/pytorch_model.bin
tokenizer config file saved in mushrafi88/T5-asr-corrector/tokenizer_config.json
Special tokens file saved in mushrafi88/T5-asr-corrector/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/944M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mushrafi88/T5-asr-corrector
   c9b56ef..c92e63f  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/mushrafi88/T5-asr-corrector
   c9b56ef..c92e63f  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/mushrafi88/T5-asr-corrector
   c92e63f..af3aaae  main -> main

   c92e63f..af3aaae  main -> main



In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mushrafi88/T5-asr-corrector")

model = AutoModelForSeq2SeqLM.from_pretrained("mushrafi88/T5-asr-corrector")

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file mushrafi88/T5-asr-corrector/config.json
Model config MT5Config {
  "_name_or_path": "mushrafi88/T5-asr-corrector",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping"

## Testing

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/bert_asr/bert_corrector/data/test/symspell_nov.csv')

In [ ]:
df_test = df_test[['sentence','wav2vec2','4gram']]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_name).to('cuda')

loading configuration file mushrafi88/T5-asr-corrector/config.json
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Model config T5Config {
  "_name_or_path": "flax-community/bengali-t5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
  

In [ ]:
from transformers import pipeline

generator = pipeline(model=model_name, tokenizer=tokenizer)

loading configuration file mushrafi88/T5-asr-corrector/config.json
Model config MT5Config {
  "_name_or_path": "mushrafi88/T5-asr-corrector",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": 

In [ ]:
df_test.wav2vec2[1500]

'তাই মসার আবাস্থ উদ্ধ্বশ করে মসাব বং বিস্তাবক্রতিরোধ করতে হবে'

In [ ]:
generator(df_test.wav2vec2[1500])

[{'generated_text': 'তাই মসরাশবসায় মসরাশবসা'}]